# TensorRTへの変換



In [ ]:
import torch
import torchvision

CATEGORIES = ['xy','speed']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

In [ ]:
from torch2trt import torch2trt

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
import time
import datetime
import subprocess

last_model_widget = ipywidgets.Dropdown(options=[],description='変換前')
last_model_time_widget = ipywidgets.Label(description='作成日時：')

trt_model_widget = ipywidgets.Text(description='変換後', value="result.pth")
convert_button = ipywidgets.Button(description='TRT Convert')


def last_model_list(change):
    global last_model_widget
    try:
        files = glob.glob('./model/*.pth', recursive=True)
        last_model_widget.options = files
        
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        last_model_time_widget.value = f'作成日時：{s}'
    except:
        last_model_widget.options = []
last_model_list("list")

def change_file(change):
    file = last_model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    last_model_time_widget.value = f'作成日時：{s}'
last_model_widget.observe(change_file, names='value')


def convert_trt(change):
    global model,model_trt,last_model_widget,trt_model_widget
    write_log("モデルの変換を開始します。")
    start_time = time.time()
    write_log("変換前: " + str(last_model_widget.value))
    model.load_state_dict(torch.load(last_model_widget.value))
    write_log("TRTへの変換処理中...(時間がかかります)")
    data = torch.zeros((1, 3, 224, 224)).cuda().half()
    model_trt = torch2trt(model, [data], fp16_mode=True)
    write_log("変換後: " + str(trt_model_widget.value))
    path = "./model_trt/"
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    torch.save(model_trt.state_dict(), path + trt_model_widget.value)
    write_log("保存成功")
    end_time = time.time() - start_time
    write_log("---------------------------------")
    write_log("処理時間　　　　　　　　　: " + str(round(end_time,2)) + "秒")
    write_log("変換前のモデル: " + str(last_model_widget.value))
    write_log("変換後のモデル: " + str(trt_model_widget.value))
    write_log("---------------------------------")
    
convert_button.on_click(convert_trt)

convert_widget = ipywidgets.VBox([
    ipywidgets.HBox([last_model_widget,last_model_time_widget]),
    ipywidgets.HBox([trt_model_widget]),
    convert_button,
    process_widget
])
display(convert_widget)